# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]          # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

KeyboardInterrupt: 

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [1]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name='Reacher.app')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [2]:
# Make all important imorts and create agent

import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from ddpg_agent import Agent


agent = Agent(state_size=33, action_size=4)

In [3]:
def learn_a2c(n_episodes: int =  300, max_actions:int = 400 ):
    scores = []
    scores_window = deque(maxlen=100)
    
    
    for i_episode in range(1,n_episodes+1):
        
        #reset the environment for every episode and initialize the state
        #step_brain_info = env.reset(train_mode=i_episode % 100 != 0)[brain_name] # reset the environment
        step_brain_info = env.reset(train_mode=True)[brain_name] # reset the environment
        
        
        state = step_brain_info.vector_observations[0]   #obtain the starting state
        
        score = 0 #initialze the score for the episode
        
        
        for a in range(max_actions):
            action = agent.act(state)
            
            step_brain_info = env.step(action)[brain_name]
            
            next_state: np.ndarray = step_brain_info.vector_observations[0]   # get the next state
            reward = step_brain_info.rewards[0]                   # get the reward
            done = step_brain_info.local_done[0]                  # see if episode has finished
            
            score += reward
            
            agent.step(state, action, reward, next_state, done)
            
            state = next_state                             # roll over the state to next time step
            if done:                             # exit loop if episode finished
                break
                
            if env.global_done:
                print('Global Done Steps: '+ str(a))
                break
        
        scores.append(score)
        scores_window.append(score)       # save most recent score
        
        print('\rEpisode {}\tavg Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tavg Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            #torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
            
    
    return scores
            
    
scores =  learn_a2c()
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show() 
    
    

np action np np !! 👇🏼
[0.10209691 0.33585557 0.01837738 0.16932458]
np action np np !! 👇🏼
[0.10253961 0.34162006 0.03497098 0.17672494]
np action np np !! 👇🏼
[0.10392849 0.34830835 0.05263969 0.17832515]
np action np np !! 👇🏼
[0.10575354 0.35684225 0.07234178 0.17569007]
np action np np !! 👇🏼
[0.10842163 0.36703554 0.09205724 0.16534182]
np action np np !! 👇🏼
[0.10966033 0.37747538 0.11019325 0.15424918]
np action np np !! 👇🏼
[0.10820203 0.3854111  0.12457162 0.13994099]
np action np np !! 👇🏼
[0.10434297 0.39064217 0.1347471  0.12512228]
np action np np !! 👇🏼
[0.09309653 0.3935329  0.12696584 0.11166617]
np action np np !! 👇🏼
[0.08245661 0.39573416 0.12198634 0.09997384]
np action np np !! 👇🏼
[0.07346685 0.39769933 0.11952464 0.091857  ]
np action np np !! 👇🏼
[0.06525505 0.3990021  0.11367548 0.08270017]
np action np np !! 👇🏼
[0.05896058 0.40016186 0.10571093 0.07334483]
np action np np !! 👇🏼
[0.05898628 0.40263784 0.09874368 0.06459939]
np action np np !! 👇🏼
[0.06173234 0.40290794 0.0

np action np np !! 👇🏼
[-0.01113045  0.2913241   0.2751218  -0.41544127]
np action np np !! 👇🏼
[-0.08228607  0.31568083  0.26640883 -0.44342184]
np action np np !! 👇🏼
[-0.10222757  0.33587644  0.23795779 -0.42218265]
np action np np !! 👇🏼
[-0.11824472  0.31441495  0.19987412 -0.35864913]
np action np np !! 👇🏼
[-0.13832434  0.26786458  0.16528238 -0.29935768]
np action np np !! 👇🏼
[-0.16920549  0.20836896  0.05324334 -0.20216139]
np action np np !! 👇🏼
[-0.11771744  0.17869142 -0.01100318 -0.15597747]
kukulo
tensor([[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0, 

tensor([[ 0.0365,  0.3934,  0.0372, -0.1501],
        [ 0.1566,  0.2890,  0.1804, -0.0580],
        [ 0.2582,  0.4909,  0.3291, -0.3164],
        [ 0.0136,  0.4290,  0.0536,  0.0685],
        [-0.3157,  0.0718, -0.1876, -0.1935],
        [-0.1419,  0.1576,  0.0177, -0.1701],
        [ 0.0090,  0.4147,  0.0135, -0.0275],
        [ 0.0484,  0.3623, -0.0179,  0.1732],
        [-0.1294,  0.1550,  0.1925, -0.1000],
        [-0.1692,  0.3161,  0.1419, -0.3460],
        [-0.1602,  0.0185,  0.2338, -0.1274],
        [ 0.1173,  0.3713,  0.0880, -0.3046],
        [ 0.0480,  0.3562, -0.0344,  0.1649],
        [ 0.0238,  0.3699,  0.0401, -0.1546],
        [ 0.0004,  0.0849,  0.0349, -0.1507],
        [-0.1365,  0.1576, -0.0236, -0.1577],
        [-0.2068,  0.0937,  0.0906, -0.2600],
        [ 0.0268,  0.0781, -0.0484, -0.0276],
        [-0.0961,  0.0999,  0.0003, -0.2408],
        [ 0.3150,  0.2927,  0.1452,  0.1693],
        [-0.1937,  0.3751,  0.1024, -0.4412],
        [-0.0030,  0.4285,  0.0417

tensor([[ 0.0643,  0.3057, -0.0356, -0.0823],
        [-0.0482,  0.4884, -0.0058, -0.1838],
        [-0.0099,  0.4151, -0.0325, -0.1541],
        [ 0.0264,  0.3397,  0.1185, -0.2563],
        [-0.1906,  0.1219, -0.1372, -0.1257],
        [-0.3541,  0.0311, -0.2226, -0.2124],
        [-0.0319,  0.4525, -0.0014,  0.0592],
        [ 0.0382,  0.1285, -0.1069,  0.0124],
        [-0.2587,  0.0575,  0.0061, -0.2803],
        [ 0.0156,  0.0960, -0.1147,  0.0081],
        [-0.0566,  0.4509, -0.0172,  0.0268],
        [-0.0261,  0.3865, -0.0182, -0.1819],
        [ 0.0440,  0.1497, -0.1071, -0.0030],
        [-0.0847,  0.4612, -0.0140, -0.1733],
        [ 0.0010,  0.4251, -0.0004,  0.1369],
        [-0.0177,  0.4501,  0.0125,  0.0767],
        [-0.0411,  0.3583,  0.0317, -0.2276],
        [ 0.2236,  0.3321,  0.3341, -0.2591],
        [-0.0315,  0.3751, -0.0234, -0.1852],
        [-0.1396,  0.4666, -0.0550, -0.2033],
        [-0.2741,  0.1840, -0.0676, -0.1839],
        [-0.1590,  0.2044,  0.1319

np action np np !! 👇🏼
[ 0.10983045  0.53003615 -0.21403717 -0.1254806 ]
np action np np !! 👇🏼
[ 0.09773304  0.5338127  -0.22145079 -0.12878849]
np action np np !! 👇🏼
[ 0.08197787  0.53409046 -0.2286578  -0.15253453]
np action np np !! 👇🏼
[ 0.05456877  0.531394   -0.21306284 -0.16836394]
np action np np !! 👇🏼
[ 0.00311844  0.51342595 -0.18815759 -0.17983966]
np action np np !! 👇🏼
[-0.08216123  0.48111573 -0.16911641 -0.21550873]
np action np np !! 👇🏼
[-0.16040292  0.4510038  -0.15540521 -0.2646083 ]
np action np np !! 👇🏼
[-0.24203065  0.414802   -0.09291443 -0.33340544]
kukulo
tensor([[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
 

...

np action np np !! 👇🏼
[-0.15792659  0.16585433 -0.59288704 -0.1477268 ]
np action np np !! 👇🏼
[-0.231861    0.23807676 -0.53064775 -0.27517608]
np action np np !! 👇🏼
[-0.2932335   0.2512366  -0.52736783 -0.38790783]
kukulo
tensor([[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0

np action np np !! 👇🏼
[-0.10156755  0.5218875  -0.43907857 -0.5687969 ]
np action np np !! 👇🏼
[-0.07164104  0.5306841  -0.40953955 -0.5953769 ]
np action np np !! 👇🏼
[-0.02130242  0.5298356  -0.35778242 -0.6011638 ]
np action np np !! 👇🏼
[-0.09570355  0.5126008  -0.33383855 -0.6235263 ]
np action np np !! 👇🏼
[-0.19209035  0.5297624  -0.38132337 -0.6644449 ]
kukulo
tensor([[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [

np action np np !! 👇🏼
[-0.3123392  0.5259304 -0.7691066 -0.6172328]
np action np np !! 👇🏼
[-0.35910046  0.50297195 -0.7423524  -0.59103876]
kukulo
tensor([[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0, 

np action np np !! 👇🏼
[-0.35090148  0.31269217 -0.82602    -0.36726698]
np action np np !! 👇🏼
[-0.45811227  0.37746394 -0.86295015 -0.33465576]
kukulo
tensor([[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0, 

KeyboardInterrupt: 

In [8]:
#Save the learned weights
agent.save_weights()

In [4]:
# Train the critic and save the weights after training

agent.restore_critic_weights()
n_episodes = 500
for i_episode in range(1,n_episodes+1):
    
    print('\rEpisode ' + str(i_episode), end="")
    #reset the environment for every episode and initialize the state
    #step_brain_info = env.reset(train_mode=i_episode % 100 != 0)[brain_name] # reset the environment
    step_brain_info = env.reset(train_mode=True)[brain_name] # reset the environment
    state = step_brain_info.vector_observations[0]   #obtain the starting state
    
    for a in range(500):
        action = agent.act(state)
            
        step_brain_info = env.step(action)[brain_name]
            
        next_state: np.ndarray = step_brain_info.vector_observations[0]   # get the next state
        reward = step_brain_info.rewards[0]                   # get the reward
        done = step_brain_info.local_done[0]                  # see if episode has finished
            
        agent.train_critic(state, reward, next_state, done)
            
        state = next_state                             # roll over the state to next time step
        if done:                             # exit loop if episode finished
            break
                
        if env.global_done:
            print('Global Done Steps: '+ str(a))
            break
    
    

agent.save_critic_weights()

Episode 500

In [ ]:
# See the trained agent in action

